In [11]:
from pdfminer.layout import LAParams, LTTextBox
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
import json
import time 
from os import listdir

import re
import spacy
import es_core_news_sm

In [13]:
nlp = spacy.load("es_core_news_sm")

OSError: [E050] Can't find model 'es_core_news_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [2]:
# Carga de docs

doc = '../data/pdfs/edicion//Unidad2_Audiolibro.pdf'

js = {}
js['name'] = f'{doc[:-4]}.json' # Apañar


In [28]:
# Parámetros del pdfminer

fp = open(doc, 'rb')
rsrcmgr = PDFResourceManager()
laparams = LAParams()
device = PDFPageAggregator(rsrcmgr, laparams=laparams)
interpreter = PDFPageInterpreter(rsrcmgr, device)
pages = PDFPage.get_pages(fp)

tuplas = {}
frases_final = []

lista_pruebas_solo_jupyter = []
lista_buena = []
lista_final = []


In [29]:
# Procesamiento de un documento

resultado = []
for i,page in enumerate(pages):
    
    # Nos saltamos los índices
    # ToDo: Permitir al usuario elegir que página saltarse
    if i == 1:
        continue
    
    tuplas[i] = []
    print('Processing next page...')
    interpreter.process_page(page)
    layout = device.get_result()
    for lobj in layout:
        if isinstance(lobj, LTTextBox):
            x, y, text = lobj.bbox[0], lobj.bbox[3], lobj.get_text()

            if y > 70:
                text = re.sub('_{2,}.+(?:\n+.+)+','',text)
                tuplas[i].append((y, text))
                #print('At %r is text: %s' % ((x, y), text))
                #print(text)
                
    prueba = []
    for a in tuplas[i]:
        prueba.append(a)
    prueba.sort(key=lambda y: ( -y[0]))
    
    pagina = []
    for a in prueba:
        
        pagina.append(' '.join(a[1].split()))
    print(pagina)
    resultado.append(pagina)    
    

Processing next page...
['UNIDAD 2: SEGUNDA PARTE', 'EL AUDIOLIBRO']
Processing next page...
['', '', 'INTRODUCCIÓN En los últimos años, los audiolibros han irrumpido con fuerza en el sector editorial. Este hecho ha desencadenado la necesidad de estudios profundo sobre el tema. Aunque en apariencia son una gran novedad, los audiolibros no son un producto completamente nuevo sino el resultado de la evolución de una serie de prácticas y soportes con un larga tradición histórica.', 'En este tema, primero, analizaremos sus orígenes y evolución con el objetivo de lograr una mayor delimitación del término. Una parte importante del interés del “fenómeno audiolibro” radica en el gran crecimiento experimentado en su uso y ventas y su impacto económico para la industria editorial. Pero, por otra parte, ha dado lugar a lo que algunos autores denominan “nueva oralidad” (u oralidad digital) y la aparición de un nuevo tipo de lector: el audiolector. En este aspecto, eldebate', 'CONCEPTO. Historia y 

['', 'INDUSTRIA DEL AUDIOLIBRO. Mercado español e iberoamericano', 'A nivel global, uno de los indicadores más evidentes del mercado del audiolibro es el crecimiento del número de títulos publicados. Según una encuesta de ventas de 2018 de los miembros de la Asociación de Editores de Audio de EE.UU., la producción de títulos entre 2014 y 2018 casi se duplicó. Por otra parte, en cuanto a los mercados anglosajones, un informe señala que las principales editoriales les han confirmado que 1 de cada 10 libros vendidos es en formato audio.', 'En concreto, HarperCollins y Hachette ya declaran que los audiolibros les suponen el 25% y 21%, respectivamente (incluyendo los ebooks en Hachette) de todos sus ingresos digitales. (Según Hurtado (2018)10: “Los audiolibros merecen una especial mención por el crecimiento que han tenido en el mercado estadounidense y las apuestas que editoriales de Iberoamérica están haciendo en la producción de contenidos en este nuevo formato…….Para el caso del mercado 

['', 'EL AUDIOLECTOR', 'Sin entrar en el debate lectura tradicional versus la escucha de un libro, sus diferencias ylas ventajas e inconvenientes de cada una de las modalidades, aquí usamos el término audiolector, recientemente popularizado, para referirnos al usuario de audiolibros.', 'Unos de los principales cambios que ha producido en el consumo de este tipo de obras tiene que ver con la ampliación de su público objetivo . La producción de audiolibros, a menudo, ha sido vista por la industria editorial como orientada a nichos de mercado muy limitados como los ciegos, o discapacitados visuales, analfabetos o personas con dificultades de lectura, ancianos y niños. Pero esto ha cambiado radicalmente, el público al que ahora se enfoca no tiene limitación alguna.', 'Entre las ventajas del uso del audiolibro más frecuentemente destacada está la posibilidad de realizar otra tarea simultáneamente a la audiolectura: pasear, limpiar la casa, viajar, etc. Esta posibilidad se deriva de la movil

In [43]:
# Post-procesado 
no_blanks = []
for pagina in resultado:
    for parrafo in pagina:
        if len(parrafo) >1:
            clean = re.sub('•','',parrafo).strip()
            no_blanks.append(clean)

In [46]:
no_blanks

['UNIDAD 2: SEGUNDA PARTE',
 'EL AUDIOLIBRO',
 'INTRODUCCIÓN En los últimos años, los audiolibros han irrumpido con fuerza en el sector editorial. Este hecho ha desencadenado la necesidad de estudios profundo sobre el tema. Aunque en apariencia son una gran novedad, los audiolibros no son un producto completamente nuevo sino el resultado de la evolución de una serie de prácticas y soportes con un larga tradición histórica.',
 'En este tema, primero, analizaremos sus orígenes y evolución con el objetivo de lograr una mayor delimitación del término. Una parte importante del interés del “fenómeno audiolibro” radica en el gran crecimiento experimentado en su uso y ventas y su impacto económico para la industria editorial. Pero, por otra parte, ha dado lugar a lo que algunos autores denominan “nueva oralidad” (u oralidad digital) y la aparición de un nuevo tipo de lector: el audiolector. En este aspecto, eldebate',
 'CONCEPTO. Historia y evolución El origen de los audiolibros se remonta a l

In [54]:
len(no_blanks)

124

In [48]:
frasecitas = []
parrafo_correspondiente = []
for p in no_blanks:
    
        for b in a.split('\n'):
            if len(b.strip().split()) > 0:
                frasecitas.append(b.strip())
                parrafo_correspondiente.append(a.strip())
    else:
        frasecitas.append(a.strip())
        parrafo_correspondiente.append(a.strip())

In [51]:
len(frasecitas)

124

In [52]:
len(parrafo_correspondiente)

124

In [ ]:
js = {}
js['name'] = 'OIM'
js['subname'] = 'Introducción'
js['paragraph'] = parrafo_correspondiente
js['content'] = frasecitas

In [ ]:
with open(f"json\\{js['name']}_{js['subname']}.json", 'w', encoding='UTF-8') as f:
    json.dump(js, f, ensure_ascii=False)